requiremenrs.txt

langchain
langchain-openai
langchainhub # langchain python라이브러리로 프롬프트, 에이전트, 체인 관련 패키지 모음
langserve[all]

faiss-cpu  # Facebook에서 개발 및 배포한 밀집 벡터의 유사도 측정, 클러스터링에 효율적인 라이브러리
tavily-python # 언어 모델에 중립적인 디자인으로, 모든 LLM과 통합이 가능하도록 설계된 검색 API
beautifulsoup4  #파이썬에서 사용할 수 있는 웹데이터 크롤링 라이브러리
wikipedia

fastapi #  Python의 API를 빌드하기 위한 웹 프레임워크
uvicorn # ASGI(Asynchronous Server Gateway Interface) 서버
urllib3 # 파이썬에서 HTTP 요청을 보내고 받는 데 사용되는 강력하고 유연한 라이브러리

python-dotenv
pypdf

In [ ]:
!pip install -r requirements.txt

Tavily Search 를 사용하기 위해서는 API KEY를 발급 받아 등록해야 함.

[Tavily Search API 발급받기](https://app.tavily.com/sign-in)

발급 받은 API KEY 를 다음과 같이 환경변수에 등록

In [ ]:
import os

from dotenv import load_dotenv

load_dotenv() 
openai_api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

# 디버깅을 위한 프로젝트명을 기입합니다.
os.environ["LANGCHAIN_PROJECT"] = "AGENT TUTORIAL"

search.invoke 함수는 주어진 문자열에 대한 검색을 실행

invoke() 함수에 검색하고 싶은 검색어를 넣어 검색을 수행

In [ ]:
# TavilySearchResults 클래스를 langchain_community.tools.tavily_search 모듈에서 가져옵니다.
from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=5은 검색 결과를 5개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=5)

In [ ]:
# 검색 결과를 가져옵니다.
search.invoke("판교 카카오 프렌즈샵 아지트점의 전화번호는 무엇인가요?")

#PDF 기반 문서 검색 도구: Retriever

내부 데이터에 대해 조회를 수행할 retriever 생성.

*   웹 기반 문서 로더, 문서 분할기, 벡터 저장소, 그리고 OpenAI 임베딩을 사용하여 문서 검색 시스템을 구축
*   PDF 문서를 FAISS DB 에 저장하고 조회하는 retriever 를 생성



In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("/content/SPRI_AI_Brief_2023년12월호_F.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

embedding_model = OpenAIEmbeddings()
# VectorStore를 생성합니다.
vector = FAISS.from_documents(split_docs, embedding_model)

# Retriever를 생성합니다.
retriever = vector.as_retriever()

retriever 객체의 get_relevant_documents 메소드를 사용

In [ ]:
# PDf 문서에서 Query 에 대한 관련성 높은 Chunk 를 가져옵니다.
retriever.get_relevant_documents(
    "YouTube의 2024년부터 AI 생성콘텐츠 표시 의무화에 대한 내용을 알려줘"
)[0]

create_retriever_tool 함수는 langchain 라이브러리의 tools.retriever 모듈에서 가져온다. 이 함수는 특정 데이터를 검색하기 위한 도구를 생성하는 데 사용된다. langchain은 언어 모델과 관련된 다양한 기능을 제공하는 라이브러리로, 이 중 검색 도구 생성 기능은 데이터 검색 및 처리 작업을 용이하게 한다.

In [ ]:
# langchain 패키지의 tools 모듈에서 retriever 도구를 생성하는 함수를 가져옵니다.
from langchain.tools.retriever import create_retriever_tool

In [ ]:
retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    description="2023년 12월 AI 관련 정보를 PDF 문서에서 검색합니다. '2023년 12월 AI 산업동향' 과 관련된 질문은 이 도구를 사용해야 합니다!",
)

# Agent 가 사용할 도구 목록 정의

이제 두 가지를 모두 만들었으므로, Agent 가 사용할 도구 목록을 만들 수 있습니다.

tools 리스트는 search와 retriever_tool을 포함합니다.

이 리스트는 검색 및 정보 검색 도구를 저장하는 데 사용됩니다.

각 요소는 특정 작업을 수행하는 데 필요한 기능을 제공합니다.

In [ ]:
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

# 에이전트 생성

이제 도구를 정의했으니 에이전트를 생성할 수 있습니다.

OpenAI Functions 에이전트를 사용할 것입니다.

먼저, 에이전트가 활용할 LLM을 정의합니다.

ChatOpenAI 클래스는 langchain_openai 모듈에서 가져온 것으로, OpenAI의 언어 모델을 활용하여 대화형 AI를 구현할 수 있게 해줍니다.

이 예제에서는 gpt-4-turbo-preview 모델을 사용하며, temperature 매개변수를 0으로 설정하여 예측의 변동성을 최소화합니다.

In [ ]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 클래스를 langchain_openai 모듈에서 가져옵니다.
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [ ]:
from langchain import hub

# hub에서 prompt를 가져옵니다 - 이 부분을 수정할 수 있습니다!
prompt = hub.pull("hwchase17/openai-functions-agent")

# prompt 의 messages를 출력합니다.
prompt.messages

 LLM, 프롬프트 및 도구로 에이전트를 초기화

에이전트는 입력을 받아 어떤 Action 을 취할지 결정하는 역할을 수행하고 Action 들을 실행하는 것은 AgentExecutor(다음 단계)에 의해 수행

In [ ]:
from langchain.agents import create_openai_functions_agent

# OpenAI 함수 기반 에이전트를 생성합니다.
# llm, tools, prompt를 인자로 사용합니다.
agent = create_openai_functions_agent(llm, tools, prompt)

에이전트(agent)를 AgentExecutor 내부의 도구들과 결합(이는 반복적으로 에이전트를 호출하고 도구들을 실행할 것임).

이 코드는 langchain.agents 모듈에서 AgentExecutor 클래스를 가져와 인스턴스를 생성.

생성 시, agent, tools 객체를 인자로 전달하고, verbose=True를 설정하여 상세한 로그 출력을 활성화.

AgentExecutor는 주어진 에이전트와 도구들을 사용하여 작업을 실행하는 역할 수행.

In [ ]:
from langchain.agents import AgentExecutor

# AgentExecutor 클래스를 사용하여 agent와 tools를 설정하고, 상세한 로그를 출력하도록 verbose를 True로 설정합니다.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 에이전트 실행하기

agent_executor 객체의 invoke 메소드는 딕셔너리 형태의 인자를 받아 처리
.

In [ ]:
# 'agent_executor' 객체의 'invoke' 메소드를 호출하여,
# 'input' 키와 '안녕, 반가워' 값을 가진 딕셔너리를 인자로 전달합니다.
response = agent_executor.invoke({"input": "2024년 아시안컵 대한민국의 축구 경기 결과를 알려줘."})
print(f'답변: {response["output"]}')

In [ ]:
# 'agent_executor' 객체의 'invoke' 메소드를 호출하여, 'langsmith'가 테스팅에 어떻게 도움을 줄 수 있는지에 대한 질문을 입력으로 제공합니다.
response = agent_executor.invoke(
    {
        "input": "YouTube 2024년부터 AI 생성콘텐츠 표시 의무화에 대한 내용을 PDF 문서에서 알려줘"
    }
)
print(f'답변: {response["output"]}')

In [ ]:
# 검색 결과를 요청 후 질문에 대한 답변을 출력합니다.
response = agent_executor.invoke(
    {"input": "판교 카카오 프렌즈샵 아지트점의 전화번호를 검색하여 결과를 알려주세요."}
)
print(f'답변: {response["output"]}')

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0,  # 창의성 0으로 설정
                 model_name='gpt-4o-mini',  # 모델명
                )

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

tools = load_tools(["wikipedia", "llm-math"], llm=llm) #llm-math의 경우 나이 계산을 위해 사용
agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         description='위키피이아에서 정보를 검색하고 계산이 필요할 때 사용',
                         verbose=True)


agent.run("gpt-4o는 언제 출시되었어?")

#agent.run("에드 시런이 태어난 해는? 2024년도 현재 에드 시런은 몇 살?")

In [ ]:
# 필요한 모듈 import
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.tools.retriever import create_retriever_tool
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

########## 1. 도구를 정의합니다 ##########

### 1-1. Search 도구 ###
# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=5은 검색 결과를 5개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=5)

### 1-2. PDF 문서 검색 도구 (Retriever) ###
# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("SPRI_AI_Brief_2023년12월호_F.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성합니다.
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성합니다.
retriever = vector.as_retriever()

# langchain 패키지의 tools 모듈에서 retriever 도구를 생성
retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    # 도구에 대한 설명을 자세히 기입해야 합니다!!!
    description="2023년 12월 AI 관련 정보를 PDF 문서에서 검색합니다. '2023년 12월 AI 산업동향' 과 관련된 질문은 이 도구를 사용해야 합니다!",
)

### 1-3. tools 리스트에 도구 목록을 추가합니다 ###
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

########## 2. LLM 을 정의합니다 ##########
# LLM 모델을 생성합니다.
llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0)

########## 3. Prompt 를 정의합니다 ##########

# hub에서 prompt를 가져옵니다 - 이 부분을 수정할 수 있습니다!
prompt = hub.pull("hwchase17/openai-functions-agent")
print(prompt)
########## 4. Agent 를 정의합니다 ##########

# OpenAI 함수 기반 에이전트를 생성합니다.
# llm, tools, prompt를 인자로 사용합니다.
agent = create_openai_functions_agent(llm, tools, prompt)

########## 5. AgentExecutor 를 정의합니다 ##########

# AgentExecutor 클래스를 사용하여 agent와 tools를 설정하고, 상세한 로그를 출력하도록 verbose를 True로 설정합니다.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

########## 6. 채팅 기록을 수행하는 메모리를 추가합니다. ##########

# 채팅 메시지 기록을 관리하는 객체를 생성합니다.
message_history = ChatMessageHistory()

# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대부분의 실제 시나리오에서 세션 ID가 필요하기 때문에 이것이 필요합니다
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    lambda session_id: message_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

########## 7. 질의-응답 테스트를 수행합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.invoke(
    {
        "input": "YouTube 2024년부터 AI 생성콘텐츠 표시 의무화에 대한 내용을 PDF 문서에서 알려줘"
    },
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "MyTestSessionID"}},
)
print(f"답변: {response['output']}")